In [73]:
#! pip install beautifulsoup4, requests
# !pip install selenium
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import numpy as np
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support import ui
import os
import sys
os.path.dirname(sys.executable)
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os

In [95]:
url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Milwaukee%2C%20WI&start=230"
html = requests.get(url)
html.content[0:500]

b'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel='

In [96]:
# Making the soup
from bs4 import BeautifulSoup
soup = BeautifulSoup(html.content, 'lxml')

In [150]:
# Getting everything also for multiple pages (here, for page 2, 3, 4, 5, and 6)
url_yelp = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Milwaukee%2C%20WI&start="


# we initialize the links
full_links = []
full_restaurants = []

for i in range(0,240,10):
    
    html = requests.get(url_yelp+str(i))
    
    soup = BeautifulSoup(html.content, 'lxml')
    link = soup.select(".css-166la90")
    
    # Getting the restaurant names and the restaurant links in solation
    name_restaurant = []
    link_restaurant = []

   
    for l in link:
        name_restaurant.append(l.string)
        link_restaurant.append(l.get('href'))
     
    #Here we only ectract the first 10 entries
    full_restaurants.extend(name_restaurant[0:10])
    full_links.extend(link_restaurant[0:10])
    
print(full_restaurants)

['Swingin’ Door Exchange', 'Story Hill BKC', 'Mad Rooster Cafe', 'Maxie’s', 'View MKE', 'Tusk', 'Triciclo Peru', 'St Paul Fish Company', 'Lazo’s Taco Shack', 'Five O’Clock Steakhouse', 'Blue Bat Kitchen and Tequilaria', 'Odd Duck', 'Public Table', 'Third Coast Provisions', 'Fool’s Errand', 'The Capital Grille', 'SweetDiner', 'Blue’s Egg - Milwaukee', 'Amilinda', 'Toast', 'Harbor House', 'La Merenda', 'Sobelman’s Pub & Grill', 'Uncle Wolfie’s Breakfast Tavern', 'Smoke Shack', 'Tangled Noodles', 'Momo Mee', 'Camino', 'The Tandem', 'MOVIDA - Hotel Madrid', 'Char’d', 'DanDan', 'The Noble', 'Le Reve Patisserie & Café', 'Tavo’s Signature Cuisine', 'Ca’Lucchenzo', 'Lowcountry Milwaukee', 'Morel', 'The Diplomat', 'The Explorium Brewpub Third Ward', 'Oscar’s Pub & Grill', 'Bavette', 'San Giorgio Pizzeria Napoletana', 'Buckley’s Restaurant & Bar', 'Stella Van Buren', 'Oak Barrel Public House', 'The Fitz', 'The Chef’s Table', 'Tied House Milwaukee', 'Café Benelux', 'Birch + Butcher', 'The Origina

In [151]:
len(full_restaurants)

240

In [152]:
#then we zip it together and store it in a dataframe
entry_list = list(zip(full_restaurants, full_links))
            
df_mainlinks = pd.DataFrame(entry_list,columns=['restaurant', 'link'])

In [154]:
df_mainlinks.to_csv('mainlinks.csv')

In [3]:
df_new = pd.read_csv('mainlinks.csv')

In [5]:
df_new

,Unnamed: 0,restaurant,link
0,0,Swingin’ Door Exchange,/biz/swingin-door-exchange-milwaukee?osq=Resta...
1,1,Story Hill BKC,/biz/story-hill-bkc-milwaukee?osq=Restaurants
2,2,Mad Rooster Cafe,/biz/mad-rooster-cafe-west-milwaukee?osq=Resta...
3,3,Maxie’s,/biz/maxies-milwaukee?osq=Restaurants
4,4,View MKE,/biz/view-mke-milwaukee?osq=Restaurants
...,...,...,...
235,235,Valley Inn,/biz/valley-inn-milwaukee?osq=Restaurants
236,236,Rice N Roll Bistro,/biz/rice-n-roll-bistro-milwaukee?osq=Restaurants
237,237,Crave Cafe,/biz/crave-cafe-shorewood?osq=Restaurants
238,238,Deli 1614,/biz/deli-1614-milwaukee?osq=Restaurants


In [6]:
#let's us regex to clean the df
import re

In [7]:
#we convert everyting to strings, and then exclude those rows where the column Links contains a https
df_new['restaurant'] = df_new.restaurant.astype('str') 
df_new['link'] = df_new.link.astype('str') 

In [8]:
'''then we complete the links colum for the full URL'''

df_new['Complete_links']= df_new['link'].apply(lambda x:"https://www.yelp.com"+str(x))

In [9]:
df_new.pop('link')

0      /biz/swingin-door-exchange-milwaukee?osq=Resta...
1          /biz/story-hill-bkc-milwaukee?osq=Restaurants
2      /biz/mad-rooster-cafe-west-milwaukee?osq=Resta...
3                  /biz/maxies-milwaukee?osq=Restaurants
4                /biz/view-mke-milwaukee?osq=Restaurants
                             ...                        
235            /biz/valley-inn-milwaukee?osq=Restaurants
236    /biz/rice-n-roll-bistro-milwaukee?osq=Restaurants
237            /biz/crave-cafe-shorewood?osq=Restaurants
238             /biz/deli-1614-milwaukee?osq=Restaurants
239    /biz/riverfront-pizzeria-bar-and-grill-milwauk...
Name: link, Length: 240, dtype: object

In [10]:
len(df_new) 

240

In [12]:
# we see that there are two duplicate restaurants, which we can drop subsequnetly
df_new.groupby(['restaurant']).size().sort_values(ascending=False)

restaurant
Camino                    2
Egg & Flour Pasta Bar     2
2Mesa Mexican Eatery      1
Rice N Roll Bistro        1
Richard’s Cafe            1
                         ..
Fuel Cafe 5th             1
GYRO MKE                  1
Goodkind                  1
Guadalajara Restaurant    1
Zocalo                    1
Length: 238, dtype: int64

In [76]:
df_new.drop_duplicates(keep='first', inplace=True)

In [78]:
df_new.to_csv('links.csv',index=False,header=True,encoding='utf8')

### Next, we scrape all the details about the restaurants with a web driver and the selenium package

In [85]:
df = pd.read_csv('links.csv')

In [86]:
df.tail()

,Restaurant,Complete_links
205,La Fondita De Milwaukee,https://www.yelp.com/biz/la-fondita-de-milwauk...
206,Taqueria Buenavista,https://www.yelp.com/biz/taqueria-buenavista-m...
207,Michael’s Family Restaurant,https://www.yelp.com/biz/michaels-family-resta...
208,Triskele’s,https://www.yelp.com/biz/triskeles-milwaukee?o...
209,El Beso Milwaukee - Mexican Restaurante & Cantina,https://www.yelp.com/biz/el-beso-milwaukee-mex...


In [140]:
'''options to avoide that actual windows are opened every time. create headless driver options'''
options = webdriver.FirefoxOptions()

#these options are set so that a browser doesn't actually open everytime we scrape
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

In [5]:
Links = df.Complete_links.to_list() #in the formula below Links is equal to Li
Restaurants = df.Restaurant.to_list() #in the formula below Restaurants is equal to Rest

In [141]:
''' Here we can make a little test run'''

time.sleep(3)
driver = webdriver.Firefox() #options=options 
driver.get('https://www.yelp.com/biz/highlands-cafe-wauwatosa?osq=Restaurants')

driver.maximize_window()
time.sleep(3)

#we first need to get rid of the privacy concern button so we click it
driver.find_element_by_css_selector('.onetrust-lg').click()

#then we wait until that click is processed, before we click the button where all the amenties are extended
time.sleep(3)
try:
    driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(6) > div:nth-child(2) > button:nth-child(2)').click()
except:
     print('no hidden amenities')

try:
    driver.find_element_by_css_selector(' div.padding-t3__373c0__1gw9E:nth-child(3) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1)').click()
    time.sleep(3)
    driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(5) > div:nth-child(2) > button:nth-child(2)').click()
    time.sleep(2)
    print('child 5')
    
except:
    print('no ammenities')
    

source = driver.page_source
soup1 = BeautifulSoup(source, 'html.parser')

no ammenities


In [24]:
driver.quit()

In [25]:
Li = df_new.Complete_links.to_list()
Rest = df_new.restaurant.to_list()

In [30]:
'''Selenium search automation can either fetch the search item from the list or text can be entered'''

df_rest = pd.DataFrame([],columns=['restaurant', 'link', 'claimed', 'number_of_reviews', 'number_of_photos', 'price', 'rating', 'categories', 'street', 'address', 'website', 'update', 'amenities'])


#This is so we can enumerate over the restaurant names
enum_extract = enumerate(Rest)

def getname():
    key = next(enum_extract)
    return str(key)

enum_rest = enumerate(Li)

def getlink():
    key = next(enum_rest)
    return str(key)
    


for i in Li:
    restaurant_link = []
    restaurant_update = []
    restaurant_description = []
    restaurant_numberofreviews = []        
    restaurant_photos = []
    restaurant_rating = []
    restaurant_street = []
    restaurant_adress = []

    restaurant_link = []
    restaurant_price = []
    restaurant_categories = []
    restaurant_name = [] 

    restaurant_claimed = [] 
    restaurant_amenities = []

    #so here we append the restaurant name
    restname = getname()
    restaurant_name.append(restname) 
    
    restlink = getlink()
    restaurant_link.append(restlink)
    
    try:
            #Here we initialize a headless driver
        time.sleep(3)
        driver = webdriver.Firefox(options=options) #options=options 
        driver.get(i)

        driver.maximize_window()
        time.sleep(3)

        #we first need to get rid of the privacy concern button so we click it
        driver.find_element_by_css_selector('.onetrust-lg').click()

        #then we wait until that click is processed, before we click the button where all the amenties are extended
        time.sleep(3)
        
        try:
            driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(6) > div:nth-child(2) > button:nth-child(2)').click()
            time.sleep(3)
        except:
            print('no hidden amenities')
        
        try:
            driver.find_element_by_css_selector(' div.padding-t3__373c0__1gw9E:nth-child(3) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1)').click()
            time.sleep(3)
            driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(5) > div:nth-child(2) > button:nth-child(2)').click()
            time.sleep(2)
            print('child 5')
        
        except:
            print('no ammenities')


        source = driver.page_source
        soup1 = BeautifulSoup(source, 'html.parser')


        #now we loop over the elements in the class
        subsoup = soup1.find_all(class_ = 'arrange-unit__373c0__1piwO border-color--default__373c0__2oFDT')
        amenities = []
        for k in range(len(subsoup)):
            subamenities=subsoup[k].get_text()
            amenities.append(subamenities)

        restaurant_amenities.append(amenities) 


        #number of reviews
        numberofreviews = soup1.select(".css-bq71j2") 
        restaurant_numberofreviews.append(numberofreviews[0].string)

        #number of photos
        photos = soup1.select(".css-ardur")
        restaurant_photos.append(photos[4].string) 

        #first and second address line
        location = soup1.select(".raw__373c0__3rcx7")
        restaurant_street.append(location[0].string) 
        restaurant_adress.append(location[1].string)

            #rating
        for item in soup1.select('[class*=container]'):
            if item.find('h4'):
                name = item.find('h4').get_text()
                ratings=soup1.select('[aria-label*=rating]')[0]
                rating=str(ratings)
                rating = re.search('aria-label="(.+?)star rating"', rating)
                if rating:
                    rating = rating.group(1)

        restaurant_rating.append(rating)

        #price
        price = soup1.select(".css-1xxismk")
        price=str(price)
        price = re.search('"css-1xxismk">(.+?)<', price)
        if price:
            price = price.group(1)
            restaurant_price.append(price) 

        #categories
        #this list needs to be inside the loop
        categories = []

        for item in soup1.select('[class*=container]'):
            if item.find('h1'):
                items=item.select('.css-166la90')
                for i in items:
                    categories.append(i.string)

                restaurant_categories.append(categories)

        if 'Unclaimed' not in categories:

            #updates
            update=soup1.select('.css-gdi06s')
            restaurant_update.append(update[0].string)             


            #homepage link
            link = soup1.select(".css-ac8spe")
            restaurant_link.append(link[6].string)


            restaurant_claimed.append('Claimed')

        else: 
            restaurant_update.append('None')
            restaurant_link.append('None')
            restaurant_claimed.append('Unclaimed')

            driver.quit()


    except: 
        restaurant_claimed.append('None')
        restaurant_update.append('None')
        restaurant_numberofreviews.append('None')
        restaurant_photos.append('None')
        restaurant_rating.append('None')
        restaurant_street.append('None')
        restaurant_adress.append('None')
        restaurant_link.append('None')
        restaurant_price.append('None')
        restaurant_categories.append('None')
        restaurant_amenities.append('None')
            
    finally:
        if len(restaurant_claimed) == 0:
            restaurant_claimed.append('None')
                
        if len(restaurant_update) == 0:
            restaurant_update.append('None')
                
        if len(restaurant_numberofreviews) == 0:
            restaurant_numberofreviews.append('None')
                
        if len(restaurant_photos) == 0:
            restaurant_photos.append('None')
                
        if len(restaurant_rating) == 0:
            restaurant_rating.append('None')
                
        if len(restaurant_street) == 0:
            restaurant_street.append('None')
                
        if len(restaurant_adress) == 0:
            restaurant_adress.append('None')
                
        if len(restaurant_link) == 0:
            restaurant_link.append('None')
                
        if len(restaurant_price) == 0:
            restaurant_price.append('None')
                
        if len(restaurant_categories) == 0:
            restaurant_categories.append('None')
                
        if len(restaurant_amenities) == 0:
            restaurant_amenities.append('None')
                
        entry_list = list(zip(restaurant_name, restaurant_link, restaurant_claimed, restaurant_numberofreviews, restaurant_photos, restaurant_price, restaurant_rating, restaurant_categories, restaurant_street, restaurant_adress, restaurant_link,restaurant_update, restaurant_amenities))

            
        df_entry = pd.DataFrame(entry_list,columns=['restaurant', 'link', 'claimed', 'number_of_reviews', 'number_of_photos', 'price', 'rating', 'categories', 'street', 'address', 'website', 'update', 'amenities'])
            
        frames = [df_rest, df_entry]
        df_rest = pd.concat(frames)
        time.sleep(5)

no hidden amenities
no ammenities
no ammenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no ammenities
no ammenities
no ammenities
child 5
no ammenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
child 5
no ammenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no hidden amenities
no ammenities


In [103]:
#with this code we can always check whether we got blocked by yelp, the status code should be equal to 200
res  = requests.get('https://www.yelp.com/biz/lazos-taco-shack-milwaukee?osq=Restaurants')

res.status_code

200

### We scraped the restaurants in several gos, which is why we now have to concatinate the different frames and clean them up

In [32]:
df_rest.tail()

,restaurant,link,claimed,number_of_reviews,number_of_photos,price,rating,categories,street,address,website,update,amenities
0,"(30, 'Skyway Cafe')","(30, 'https://www.yelp.com/biz/skyway-cafe-mil...",Unclaimed,13 reviews,See 23 photos,$,4.5,"[Unclaimed, Cafes]",626 East Wisconsin Ave,Ground floor,"(30, 'https://www.yelp.com/biz/skyway-cafe-mil...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(31, 'The Green Kitchen')","(31, 'https://www.yelp.com/biz/the-green-kitch...",Unclaimed,164 reviews,See 77 photos,$,4,"[Unclaimed, Vegetarian, Sandwiches]",400 N Water St,"Milwaukee, WI 53202","(31, 'https://www.yelp.com/biz/the-green-kitch...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(32, 'Thainamite - Wauwatosa')","(32, 'https://www.yelp.com/biz/thainamite-wauw...",Claimed,141 reviews,See 134 photos,$$,3.5,"[Thai, Sushi Bars, Cocktail Bars]",8725 W North Ave,"Wauwatosa, WI 53226","(32, 'https://www.yelp.com/biz/thainamite-wauw...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(33, 'Deli 1614')","(33, 'https://www.yelp.com/biz/deli-1614-milwa...",Unclaimed,17 reviews,See 12 photos,$,4.5,"[Unclaimed, Delis]",1614 W Walnut St,"Milwaukee, WI 53205","(33, 'https://www.yelp.com/biz/deli-1614-milwa...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(34, 'Riverfront Pizzeria Bar & Grill')","(34, 'https://www.yelp.com/biz/riverfront-pizz...",None,136 reviews,See 110 photos,$$,3.5,"[Pizza, Bars]",509 E Erie St,"Milwaukee, WI 53202","(34, 'https://www.yelp.com/biz/riverfront-pizz...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."


In [34]:
df10 = df_rest

In [35]:
df10.to_csv('usable10.csv')

In [36]:
df1 = pd.read_csv('usable.csv')
df2 = pd.read_csv('usable2.csv')
df3 = pd.read_csv('usable3.csv')
df4 = pd.read_csv('usable4.csv')
df5 = pd.read_csv('usable5.csv')
df6 = pd.read_csv('usable6.csv')
df7 = pd.read_csv('usable7.csv')
df8 = pd.read_csv('usable8.csv')
df9 = pd.read_csv('usable9.csv')
df10 = pd.read_csv('usable10.csv')

In [48]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]
dfdf = pd.concat(frames)

In [49]:
len(dfdf)

297

In [50]:
dfdf.head()
dfdf = dfdf.loc[:, ~dfdf.columns.str.contains('^Unnamed')]
dfdf['restaurant'] = dfdf['restaurant'].astype('str') 

In [53]:
import regex as re

new_restaurants = dfdf.restaurant.to_list()

'''stripping anything werid from the search term'''
def searchconcat(x):
    x = x.replace(',', '')\
    .replace("'","")\
    .replace('(','')\
    .replace(')','')\
    .replace('0', '')\
    .replace('1', '')\
    .replace('2', '')\
    .replace('3', '')\
    .replace('4', '')\
    .replace('5', '')\
    .replace('6', '')\
    .replace('7', '')\
    .replace('8', '')\
    .replace('9', '')\
    .strip()
    
    return str(x)



new_ii = []
for i in new_restaurants:
    new_i=searchconcat(i)
    new_ii.append(new_i)
    

dfdf['restaurant'] = new_ii

In [56]:
dfdf.drop_duplicates(subset=['restaurant'], keep='last', inplace = True)

In [62]:
len(dfdf)

253

### Then here is the final dataframe

In [63]:
dfdf.to_csv('milwaukee.restaurants.yelp.csv')

In [66]:
dflist = dfdf.restaurant.to_list()
len(set(dflist))

253

### Now on to the individual reviews

In [252]:
df = pd.read_csv('mainlinks.csv')

In [253]:
'''then we complete the links colum for the full URL'''

df['Complete_links']= df['link'].apply(lambda x:"https://www.yelp.com"+str(x))

In [254]:
Links = df.Complete_links.to_list() #in the formula below Links is equal to Li
Restaurants = df.restaurant.to_list() #in the formula below Restaurants is equal to Rest

In [255]:
restrest = Restaurants[:]
linklink = Links[:]

In [78]:
url_yelp = 'https://www.yelp.com/biz/sweetdiner-milwaukee?osq=Restaurants'
html = requests.get(url_yelp)
soup2 = BeautifulSoup(html.content, 'lxml')

html.status_code

200

In [112]:
import pyautogui

In [129]:
df1.to_csv('all_reviews1.csv')

In [185]:
restrest = df_again.restaurant.to_list()
linklink= df_again.Complete_links.to_list()

In [142]:
df_copy = df1.copy()

In [258]:
len(linklink), len(restrest)

(240, 240)

In [270]:
df4.to_csv('reviews_63')

In [277]:
restrest = Restaurants[64:]
linklink = Links[64:]

In [278]:
''' Now we scrape the reviews, this time we do not have to make use of the selenoim package, we onyl work with URLs which should make this process a bit easier'''

#we initialize a dataframe, but of course outside the loop
df5 = pd.DataFrame([],columns=['Restaurant', 'Review'])

enum_extract = enumerate(restrest)


failedurls = []

def getname():
    key = next(enum_extract)
    return str(key)


for l in linklink:
    url_yelp = l
    html = requests.get(url_yelp)
    
    #soup2 is the base website
    soup2 = BeautifulSoup(html.content, 'lxml')
    restname = getname() 
    if html.status_code == 200:
        print('status passed')
        length = ''
        length1 = 0
        length2 = 0
        reviews = []
        try: 
            ## we search for the number of sites to loop through on the base website soup2
            length=soup2.select('.css-e81eai')
            length = str(length)
            #we use a regular expression to filter out the total length of the review
            length = re.search('>(\d+) of (\d+)<', length)
            if length:
                length1 = length.group(1)
                length2 = int(length.group(2))
                
            if length2 > 20:
                length2 = 20
            
            print(length2, length1)

            
            for i in range(0,length2*10,10):
                html = requests.get(url_yelp+'&start='+str(i))
                
                if html.status_code == 200:
                    soup3 = BeautifulSoup(html.content, 'lxml')
                    soupstring = str(soup3)
                    #this is the pattern for all the reviews, it will find all matches so we don't have to worry about not grabbing one. Though we will have to clean this up later
                    subsoup = re.findall('"markupDisplayName":(.+?),"voterText":', soupstring)
                    reviews += subsoup
                    time.sleep(5)
                else:
                    print(f'We got blocked inside a subpage {url_yelp}')
                    failedurls.append(url_yelp)
                    time.sleep(5)
                    pyautogui.alert(text='Enter password', title='Enter password', button='OK')
                    print('continue')
                    time.sleep(6)
                    
            
            #outside of the last loop, we create a df, where we zip the lists, and then we append them to the df
            keys = [restname]*int(len(reviews))
            review_list = list(zip(keys, reviews))

            df2=pd.DataFrame(review_list, columns=['Restaurant','Review'])

            frames= [df5, df2]
            df5 = pd.concat(frames)
            print(restname)
            time.sleep(3)  
                   
        except:
            length = ''
            length1 = 0
            length2 = 0
            reviews = []
            print(f'No reviews for {restname}')
            time.sleep(5)
              
        #In case we have lots of reviews, we just scrape the first 200
         
    else:
        length = ''
        length1 = 0
        length2 = 0
        reviews = []
        print(f'We got blocked for {restname} {url_yelp}')
        failedurls.append(url_yelp)
        pyautogui.alert(text='Enter password', title='Enter password', button='OK')
        print('continue')

        time.sleep(6)

status passed
12 1
(0, 'Spicy Tuna')
status passed
20 1
(1, 'Milwaukee Brat House')
status passed
20 1
(2, 'La Casa De Alberto Mexican Food')
status passed
20 1
(3, 'Ono Kine Grindz')
status passed
20 1
(4, 'Engine Company No. 3')
status passed
2 1
(5, 'On The Bayou')
status passed
1 1
(6, 'ARIA')
status passed
8 1
(7, 'Daddys Soul Food Grille')
status passed
20 1
(8, 'La Masa Empanada Bar')
status passed
5 1
(9, 'Eldr+Rime')
status passed
7 1
(10, 'Orenda Cafe')
status passed
5 1
(11, '2Mesa Mexican Eatery')
status passed
3 1
(12, 'Cavas')
status passed
9 1
(13, 'Buckatabon Tavern & Supper Club')
status passed
1 1
(14, 'Honcho Truck MKE')
status passed
20 1
(15, 'Black Sheep')
status passed
8 1
We got blocked inside a subpage https://www.yelp.com/biz/bowls-to-go-milwaukee?osq=Restaurants
continue
(16, 'Bowls To Go')
status passed
20 1
We got blocked inside a subpage https://www.yelp.com/biz/calderone-club-milwaukee-3?osq=Restaurants
continue
(17, 'Calderone Club')
status passed
16 1
(

In [279]:
len(df5)

18596

In [198]:
df1 = pd.read_csv('all_reviews1.csv')
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

In [280]:
df1 = pd.concat([df4, df5])

In [281]:
len(df1)

28391

In [282]:
import regex as re

new_restaurants = df1.Restaurant.to_list()

'''stripping anything werid from the search term'''
def searchconcat(x):
    x = x.replace(',', '')\
    .replace("'","")\
    .replace('(','')\
    .replace(')','')\
    .replace('0', '')\
    .replace('1', '')\
    .replace('2', '')\
    .replace('3', '')\
    .replace('4', '')\
    .replace('5', '')\
    .replace('6', '')\
    .replace('7', '')\
    .replace('8', '')\
    .replace('9', '')\
    .strip()
    
    return str(x)



new_ii = []
for i in new_restaurants:
    new_i=searchconcat(i)
    new_ii.append(new_i)
    

df1['Restaurant'] = new_ii

In [283]:
df1.head()

,Restaurant,Review
0,Swingin’ Door Exchange,"""KC and Shelly S."",""photoSrc"":""https://s3-medi..."
1,Swingin’ Door Exchange,"""Hart P."",""displayLocation"":""Gallatin, TN"",""al..."
2,Swingin’ Door Exchange,"""Warren D."",""displayLocation"":""Milwaukee, WI"",..."
3,Swingin’ Door Exchange,"""Daniela B."",""displayLocation"":""Brookfield, WI..."
4,Swingin’ Door Exchange,"""Ashley B."",""displayLocation"":""Chicago, IL"",""a..."


In [284]:
''' Lets cut out the photos already, cause that would be too much to parse'''

#df1[df1['Restaurant'] == 'Swingin’ Door Exchange']['Review'].values[8]

' Lets cut out the photos already, cause that would be too much to parse'

In [285]:
df1['username'] = df1['Review'].apply(lambda x: re.findall('"(.+?)"', x)[0])

In [286]:
df1['text'] = df1['Review'].apply(lambda x: re.findall('"comment":{"text":(.+?),"language":"', x)[0])

In [287]:
df1['rating'] = df1['Review'].apply(lambda x: re.findall('"rating":(.+?),', x)[0]) #"photos"

In [288]:
df1['userfeedback'] = df1['Review'].apply(lambda x: re.findall('"feedback":{"counts":{(.+?)},', x)[0])

df1['userfeedback_useful'] = df1['userfeedback'].apply(lambda x: re.findall('"useful":(.+?),', x)[0])
df1['userfeedback_funny'] = df1['userfeedback'].apply(lambda x: re.findall('"funny":(.+?),', x)[0])
df1['userfeedback_cool'] = df1['userfeedback'].apply(lambda x: re.findall('"cool":(.+?)', x)[0])

In [289]:
df1['reviewtotalPhotos'] = df1['Review'].apply(lambda x: re.findall('"totalPhotos":(.+?),', x)[0])

In [290]:
df1['peerfeedback'] = df1['Review'].apply(lambda x: re.findall('"userFeedback":{(.+?)}', x)[0])


df1['peerfeedback_useful'] = df1['peerfeedback'].apply(lambda x: re.findall('"useful":(.+?),', x)[0])
df1['peerfeedback_funny'] = df1['peerfeedback'].apply(lambda x: re.findall('"funny":(.+?),', x)[0])
df1['peerfeedback_cool'] = df1['peerfeedback'].apply(lambda x: re.findall('"cool":(.+?)', x)[0])

In [291]:
df1['language'] = df1['Review'].apply(lambda x: re.findall('"language":"(.+?)"}', x)[0])

In [292]:
df1['Date'] = df1['Review'].apply(lambda x: re.findall('"localizedDate":"(.+?)","', x)[0])

In [293]:
df1['userlocation'] = df1['Review'].apply(lambda x: re.findall('"displayLocation":"(.+?)"', x)[0])

In [294]:
df1['friendCount'] = df1['Review'].apply(lambda x: re.findall('"friendCount":(.+?),', x)[0])

In [295]:
df1['photoCount'] = df1['Review'].apply(lambda x: re.findall('"photoCount":(.+?),', x)[0])

In [296]:
df1['reviewCount'] = df1['Review'].apply(lambda x: re.findall('"reviewCount":(.+?),', x)[0])

In [297]:
df1['eliteYear'] = df1['Review'].apply(lambda x: re.findall('"eliteYear":(.+?)},', x)[0])

In [298]:
def elite(x):
    x=re.findall('"isCurrentlyElite":(.+?),', x)
    try:
        if x[0]:
            return x[0]
    except:
        x ='false'
        return x

In [299]:
df1['isCurrentlyElite'] = df1['Review'].apply(lambda x: elite(x))

In [300]:
len(df1)

28391

In [26]:
dfx =df1[(df1['rating'] == '1') | (df1['rating'] == '2')] 
dfy = df1[(df1['rating'] == '3') | (df1['rating'] == '4')] 
dfz = df1[(df1['rating'] == '5') | (df1['rating'] == '6')]

frames = [dfx, dfy, dfz]

In [27]:
df1 = pd.concat(frames)

In [61]:
len(df1)

12392

In [222]:
df1['rating'] = df1['rating'].astype('float') 

In [223]:
df_unclened = df1[['Restaurant', 'username', 'Review']]

In [224]:
df_unclened.to_csv('df_uncleaned_reviews.csv')

In [301]:
df1.pop('Review')
df1.pop('userfeedback')
df1.pop('peerfeedback')

0      "useful":false,"funny":false,"cool":false
1      "useful":false,"funny":false,"cool":false
2      "useful":false,"funny":false,"cool":false
3      "useful":false,"funny":false,"cool":false
4      "useful":false,"funny":false,"cool":false
                         ...                    
134    "useful":false,"funny":false,"cool":false
135    "useful":false,"funny":false,"cool":false
136    "useful":false,"funny":false,"cool":false
137    "useful":false,"funny":false,"cool":false
138    "useful":false,"funny":false,"cool":false
Name: peerfeedback, Length: 28391, dtype: object

In [226]:
df1.Restaurant.nunique()

221

In [65]:
pwd()

'/Users/julia/Downloads'

In [302]:
df1.to_excel(r'/Users/julia/Downloads/all_reviews2.xlsx', index = False)

In [303]:
df1.to_csv('all_reviews2.csv')

In [308]:
df = pd.read_csv('mainlinks.csv')

In [309]:
df1 = pd.read_csv('all_reviews2.csv')

In [310]:
df1.Restaurant.nunique()

236

In [311]:
df.restaurant.nunique()

238

In [314]:
''' Now we need to comapre the difference'''

old_rest = df.restaurant.unique()

new_rest = df1.Restaurant.unique()


df_new = df1[~df1['Restaurant'].isin(old_rest)]

df_new.Restaurant.unique()

array(['Engine Company No.', 'Mesa Mexican Eatery', 'Uncle Bucks on rd',
       'Fuel Cafe th', 'Bar + Kitchen', 'Bottle House',
       'Tavolino - Wine Pasta & Pizza', 'Dugout', 'Deli'], dtype=object)